In [1]:
import pandas as pd

In [2]:
Dataset=pd.read_csv('CKD.csv')

In [3]:
Dataset.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hrmo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

In [4]:
Dataset.shape

(399, 25)

In [5]:
Dataset.info

<bound method DataFrame.info of            age         bp sg  al  su     rbc        pc         pcc  \
0     2.000000  76.459948  c   3   0  normal  abnormal  notpresent   
1     3.000000  76.459948  c   2   0  normal    normal  notpresent   
2     4.000000  76.459948  a   1   0  normal    normal  notpresent   
3     5.000000  76.459948  d   1   0  normal    normal  notpresent   
4     5.000000  50.000000  c   0   0  normal    normal  notpresent   
..         ...        ... ..  ..  ..     ...       ...         ...   
394  51.492308  70.000000  a   0   0  normal    normal  notpresent   
395  51.492308  70.000000  c   0   2  normal    normal  notpresent   
396  51.492308  70.000000  c   3   0  normal    normal  notpresent   
397  51.492308  90.000000  a   0   0  normal    normal  notpresent   
398  51.492308  80.000000  a   0   0  normal    normal  notpresent   

             ba         bgr  ...        pcv            wc        rc  htn   dm  \
0    notpresent  148.112676  ...  38.868902   

In [6]:
Dataset=pd.get_dummies(Dataset,dtype=int,drop_first=True)

In [7]:
Dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [8]:
Dataset.info

<bound method DataFrame.info of            age         bp  al  su         bgr          bu        sc  \
0     2.000000  76.459948   3   0  148.112676   57.482105  3.077356   
1     3.000000  76.459948   2   0  148.112676   22.000000  0.700000   
2     4.000000  76.459948   1   0   99.000000   23.000000  0.600000   
3     5.000000  76.459948   1   0  148.112676   16.000000  0.700000   
4     5.000000  50.000000   0   0  148.112676   25.000000  0.600000   
..         ...        ...  ..  ..         ...         ...       ...   
394  51.492308  70.000000   0   0  219.000000   36.000000  1.300000   
395  51.492308  70.000000   0   2  220.000000   68.000000  2.800000   
396  51.492308  70.000000   3   0  110.000000  115.000000  6.000000   
397  51.492308  90.000000   0   0  207.000000   80.000000  6.800000   
398  51.492308  80.000000   0   0  100.000000   49.000000  1.000000   

            sod       pot       hrmo  ...  pc_normal  pcc_present  ba_present  \
0    137.528754  4.627244  12.5181

In [9]:
Independent=Dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
Dependent=Dataset[['classification_yes']]

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(Independent,Dependent,test_size=0.30,random_state=0)

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

param_grid = {
    'alpha': [0.0, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
    'fit_prior': [True, False]
    
}
grid = GridSearchCV(MultinomialNB(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 14 candidates, totalling 70 fits


c:\Users\Gajalakshmi G\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': [0.0, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
                         'fit_prior': [True, False]},
             scoring='f1_weighted', verbose=3)

In [14]:
re=grid.cv_results_

In [15]:
grid_Pred=grid.predict(X_test)

In [16]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(grid_Pred,Y_test)
print(cm)

[[44 20]
 [ 1 55]]


In [17]:
from sklearn.metrics import classification_report
clf_report=classification_report(grid_Pred,Y_test)
print(clf_report)

              precision    recall  f1-score   support

           0       0.98      0.69      0.81        64
           1       0.73      0.98      0.84        56

    accuracy                           0.82       120
   macro avg       0.86      0.83      0.82       120
weighted avg       0.86      0.82      0.82       120



In [18]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

0.9315555555555557

In [19]:
table=pd.DataFrame.from_dict(re)
print(table)

    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_alpha  \
0        0.019188      0.003004         0.019362        0.002837         0.0   
1        0.012490      0.005526         0.018191        0.005771         0.0   
2        0.006479      0.001371         0.008511        0.000949         0.1   
3        0.007984      0.001217         0.011859        0.003551         0.1   
4        0.009144      0.002799         0.014407        0.002265         0.5   
5        0.009170      0.001933         0.012959        0.006017         0.5   
6        0.012162      0.009985         0.015397        0.004789         1.0   
7        0.011052      0.006382         0.014002        0.002862         1.0   
8        0.007950      0.000686         0.012368        0.001201         2.0   
9        0.007671      0.000852         0.011124        0.001264         2.0   
10       0.008576      0.001827         0.014260        0.001501         5.0   
11       0.007278      0.000872         